In [5]:
# 2021-02-06

# Comparing: pretrain+finetune for 10, 50, 150 epochs

# Settings 

# Import master csv file
input_dir = '~/Downloads/'
csv_name = 'poolfinetune2_20210206_10pm.csv'
output_dir = '~/Downloads/'

import pandas as pd
import numpy as np
import os

# Basic data cleaning
rundata = pd.read_csv(os.path.join(input_dir,csv_name))
rundata = rundata[rundata['State']=='finished']

rundata.loc[:, ['pretrain_epochs','bert_model','bert_induced','seed', 'source_file','target_file', 'target','item',
                                    'test_auprc']]

,pretrain_epochs,bert_model,bert_induced,seed,source_file,target_file,target,item,test_auprc
0,150,bert,True,2029,both,eicu,dx_depth1_unique,all,0.514208
1,150,bert,True,2029,both,mimic,dx_depth1_unique,all,0.746362
2,150,bert,True,2028,both,eicu,dx_depth1_unique,all,0.499875
3,150,bert,True,2028,both,mimic,dx_depth1_unique,all,0.748817
4,150,bert,True,2027,both,eicu,dx_depth1_unique,all,0.519452
...,...,...,...,...,...,...,...,...,...
1026,50,bio_clinical_bert,True,2020,both,mimic,dx_depth1_unique,all,NaN
1027,50,bio_clinical_bert,True,2020,both,mimic,los>7day,all,NaN
1028,50,bio_clinical_bert,True,2020,both,mimic,los>3day,all,NaN
1029,50,bio_clinical_bert,True,2020,both,mimic,mortality,all,NaN


In [43]:
def generate_table(result_file:pd.DataFrame, seed_count=False, count_flag=True, std=False, rounding=True):
    
    
    # Possible combinations for our models
    conditionlist = [(result_file['bert_induced'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == True),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == False)]
    choicelist = ['singleRNN', 'CLSfixed', 'CLSfinetune','BERTfinetune']
    result_file['model_name'] = np.select(conditionlist, choicelist)
    
    
    
    result_file = result_file.loc[:, ['model_name','pretrain_epochs','bert_model','bert_induced','seed', 
                                      'source_file','target_file', 'target','item','test_auprc']]
    
    result_filter = result_file.copy() #.loc[result_file['target']==target]
     
    result_mean = pd.pivot_table(result_filter,
                               columns = ['source_file', 'target_file', 'model_name'],
                               index = ['item','target','bert_induced', 'bert_model','pretrain_epochs'],
                               values = ['test_auprc'],
                               aggfunc=['mean'])
    
    result_std = pd.pivot_table(result_filter,
                               columns = ['source_file', 'target_file', 'model_name'],
                               index = ['item','target','bert_induced', 'bert_model','pretrain_epochs'],
                               values = ['test_auprc'],
                               aggfunc=['std'])
    
    result_count = pd.pivot_table(result_filter,
                               columns = ['source_file', 'target_file', 'model_name'],
                               index = ['item','target','bert_induced', 'bert_model','pretrain_epochs'],
                               values = ['test_auprc'],
                               aggfunc=['count'])
    
    if count_flag == True:
        if rounding==True:
            RM_drop = result_mean.droplevel([0], axis=1).round(3)
            RS_drop = result_std.droplevel([0], axis=1).round(3)
        elif rounding == False:
            RM_drop = result_mean.droplevel([0], axis=1)
            RS_drop = result_std.droplevel([0], axis=1)
        RC_drop = result_count.droplevel([0], axis=1)
        
        RC_drop = RC_drop[RM_drop.columns]
        
        for col in range(len(RM_drop.columns)): 
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'   
            
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'  
    
    
    if seed_count==False:
        if count_flag == True:
            if std==True:
                return RS_drop
            elif std==False:
                return RM_drop
        elif count_flag == False:
            if std==True:
                return result_std
            elif std==False:
                return result_mean
    
    elif seed_count==True:
        return RC_drop



In [44]:
keep_seed = False
count_seeds = False
flag = True

In [46]:
generate_table(rundata)

test_auprc  \
source_file                                                                 both   
target_file                                                                 eicu   
model_name                                                           CLSfinetune   
item target           bert_induced bert_model        pretrain_epochs               
all  dx_depth1_unique False        pubmed_bert       10                      NaN   
                                                     50                      NaN   
                                                     150                     NaN   
                      True         bert              150                   0.516   
                                   bio_clinical_bert 50                    0.517   
                                                     150                   0.516   
     los>3day         False        pubmed_bert       10                      NaN   
                                                     50                      NaN   
                                                     150                     NaN   
                      True         bert              50                    0.701   
                                                     150                   0.670   
                                   bio_clinical_bert 50                    0.702   
                                                     150                   0.665   
     los>7day         False        pubmed_bert       10                      NaN   
                                                     50                      NaN   
                                                     150                     NaN   
                      True         bert              50                    0.511   
                                                     150                   0.471   
                                   bio_clinical_bert 50                    0.514   
                                                     150                   0.453   
     mortality        False        pubmed_bert       10                      NaN   
                                                     50                      NaN   
                                                     150                     NaN   
                      True         bert              50                    0.188   
                                                     150                   0.143   
                                   bio_clinical_bert 50                    0.191   
                                                     150                   0.134   
     readmission      False        pubmed_bert       10                      NaN   
                                                     50                      NaN   
                                                     150                     NaN   
                      True         bert              50                    0.211   
                                                     150                   0.190   
                                   bio_clinical_bert 50                    0.185   
                                                     150                   0.175   

                                                                                \
source_file                                                                      
target_file                                                                      
model_name                                                           singleRNN   
item target           bert_induced bert_model        pretrain_epochs             
all  dx_depth1_unique False        pubmed_bert       10                  0.505   
                                                     50                  0.506   
                                                     150                 0.498   
                      True         bert              150                   NaN   
                                   bio_clinical_bert 50